In [1]:
# list of package need to import

import simpy
import numpy.random as random

In [2]:
''' ------------------------ '''
''' Parameters               '''
''' ------------------------ '''

MAXSIMTIME = 5000
VERBOSE = False
LAMBDA = 5.0
MU = 8.0
POPULATION = 50000000
SERVICE_DISCIPLINE = 'FIFO'
LOGGED = True


In [3]:

''' ------------------------ '''
''' DES model                '''
''' ------------------------ '''
class Job:
    def __init__(self, name, arrtime, duration):
        self.name = name
        self.arrtime = arrtime
        self.duration = duration

    def __str__(self):
        return '%s at %d, length %d' %(self.name, self.arrtime, self.duration)



In [4]:


''' A server
 - env: SimPy environment
 - strat: - FIFO: First In First Out
          - SJF : Shortest Job First
'''
class Server:
    def __init__(self, env, strat = 'FIFO'):
        self.env = env
        self.strat = strat
        self.Jobs = list(())
        self.serversleeping = None
        ''' statistics '''
        self.waitingTime = 0
        self.idleTime = 0
        self.jobsDone = 0
        ''' register a new server process '''
        env.process( self.serve() )

    def serve(self):
        while True:
            ''' do nothing, just change server to idle
              and then yield a wait event which takes infinite time
            '''
            yield env.timeout(2)
            
            if len( self.Jobs ) == 0 :
                self.serversleeping = env.process( self.waiting( self.env ))
                t1 = self.env.now
                yield self.serversleeping
                ''' accumulate the server idle time'''
                self.idleTime += self.env.now - t1
            else:
                ''' get the first job to be served'''
                if self.strat == 'SJF':
                    self.Jobs.sort( key = SJF )
                    j = self.Jobs.pop( 0 )
                else: # FIFO by default
                    j = self.Jobs.pop( 0 )
                if LOGGED:
                    qlog.write( '%.4f\t%d\t%d\n' 
                        % (self.env.now, 1 if len(self.Jobs)>0 else 0, len(self.Jobs)) )

                ''' sum up the waiting time'''
                self.waitingTime += self.env.now - j.arrtime
                ''' yield an event for the job finish'''
                yield self.env.timeout( j.duration )
                ''' sum up the jobs done '''
                self.jobsDone += 1

    def waiting(self, env):
        try:
            if VERBOSE:
                print( 'Server is idle at %.2f' % self.env.now )
            yield self.env.timeout( MAXSIMTIME )
        except simpy.Interrupt as i:
            if VERBOSE:
                 print('Server waken up and works at %.2f' % self.env.now )

In [5]:
class JobGenerator:
    def __init__(self, env, server, nrjobs = 10000000, lam = 5, mu = 8):
        self.server = server
        self.nrjobs = nrjobs
        self.interarrivaltime = 1/lam;
        self.servicetime = 1/mu;
        env.process( self.generatejobs(env) )

    def generatejobs(self, env):
        i = 1
        while True:
            '''yield an event for new job arrival'''
            job_interarrival = random.exponential( self.interarrivaltime )
            yield env.timeout( job_interarrival )

            ''' generate service time and add job to the list'''
            job_duration = random.exponential( self.servicetime )
            self.server.Jobs.append( Job('Job %s' %i, env.now, job_duration) )
            if VERBOSE:
                print( 'job %d: t = %.2f, l = %.2f, dt = %.2f' 
                    %( i, env.now, job_duration, job_interarrival ) )
            i += 1

            ''' if server is idle, wake it up'''
            if not self.server.serversleeping.triggered:
                self.server.serversleeping.interrupt( 'Wake up, please.' )



In [6]:
# main file 

''' open a log file '''
if LOGGED:
    qlog = open( 'mm1-l%d-m%d.csv' % (LAMBDA,MU), 'w' )
    qlog.write( '0\t0\t0\n' )

''' start SimPy environment '''
env = simpy.Environment()
MyServer = Server( env, SERVICE_DISCIPLINE )
MyJobGenerator = JobGenerator( env, MyServer, POPULATION, LAMBDA, MU )

''' start simulation '''
env.run( until = MAXSIMTIME )

''' close log file '''
if LOGGED:
    qlog.close()

''' print statistics '''
RHO = LAMBDA/MU
print( 'Arrivals               : %d' % (MyServer.jobsDone) )
print( 'Utilization            : %.2f/%.2f' 
    % (1.0-MyServer.idleTime/MAXSIMTIME, RHO) )
print( 'Mean waiting time      : %.2f/%.2f' 
    % (MyServer.waitingTime/MyServer.jobsDone, RHO**2/((1-RHO)*LAMBDA) ) )


AttributeError: 'NoneType' object has no attribute 'triggered'

In [ ]:
### need to draw somethings here

In [ ]:
"""MMC.py

An M/M/m queue

+ Jobs arrive at random into a c-server queue with
exponential service-time distribution.
+ Simulate to determine the average  number in the system and
the average time jobs spend in the system.

- c = Number of servers = 3
- rate = Arrival rate = 2.0
- stime = mean service time = 1.0

"""
import simpy
import random

NUMBER_SERVERS = 3
ARRIVAL_RATE = 2.0
SERVICE_RATE = 1.0
SERVICE_DISCIPLINE = 'FIFO'



In [ ]:
class Job:
    ''' Jobs request a gatekeeper and hold
        it for an exponential time '''
    NoInSystem = 0

    def execute(self, stime):
        arrTime = now()
        self.trace("Hello World")
        Job.NoInSystem += 1
        m.observe(Job.NoInSystem)
        yield request, self, server
        self.trace("At last    ")
        t = expovariate(1.0 / stime)
        msT.observe(t)
        yield hold, self, t
        yield release, self, server
        Job.NoInSystem -= 1
        m.observe(Job.NoInSystem)
        mT.observe(now() - arrTime)
        self.trace("Geronimo   ")

    def trace(self, message):
        FMT = "{0:7.4f} {1:6} {2:10} ({3:2d})"
        if TRACING:
            print(FMT.format(now(), self.name, message, Job.NoInSystem))

In [ ]:
class JobGenerator:
    """ generates Jobs at random """
    def __init__(self, env, server):
        self.server = server
        env.process( self.execute(env) )

    def execute(self, maxNumber, rate, stime):
        ''' generate Jobs at exponential intervals '''
        for i in range(maxNumber):
            L = Job("Job {0} ".format(i))
            activate(L, L.execute(stime), delay=0)
            yield hold, self, expovariate(rate)

In [ ]:
class Server:
    def __init__(self, env):
        self.Jobs = list(())
        env.process( self.run(env) )
    
    def run(self, env):
        self.m = Monitor(sim=self)  # monitor for the number of jobs
        self.mT = Monitor(sim=self)  # monitor for the time in system
        self.msT = Monitor(sim=self)  # monitor for the generated service times
        self.server = Resource(capacity=c, name='Gatekeeper', sim=self)
        g = Generator(name='gen', sim=self)
        self.activate(g, g.execute(maxNumber=maxNumber,
                                   rate=rate, stime=stime))
        self.m.observe(0)  # number in system is 0 at the start
        self.simulate(until=3000.0)


In [ ]:

env = simpy.Environment()
MyServer = Server( env )
MyJobGenerator = JobGenerator( env, MyServer )
env.run( until = 20 )